In [ ]:
!pip install trl

In [ ]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="ceval/ceval-exam",
    repo_type="dataset",
    revision="main",
    local_dir="/kaggle/working/data",
    local_dir_use_symlinks=False,
)

In [3]:
from datasets import load_dataset

dataset = load_dataset('parquet', data_files={
    'train': '/kaggle/working/data/*/dev-*-of-*.parquet',
    'validation': '/kaggle/working/data/*/val-*-of-*.parquet',
    'test': '/kaggle/working/data/*/test-*-of-*.parquet'
})


Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
dev_dataset = dataset['train']
validation_dataset = dataset['validation']
train_dataset = dataset['test']

In [5]:
print(train_dataset[555])

{'id': 112, 'question': '求不定积分：$I=\\int{\\frac{\\mathrm{d}x}{\\left(x^2+x+1\\right)^2}}$____', 'A': '$\\frac{1\\sqrt{3}}{9}t+\\frac{1\\sqrt{3}}{9}\\sin t\\cos t+C$', 'B': '$\\frac{2\\sqrt{3}}{9}t+\\frac{2\\sqrt{3}}{9}\\sin t\\cos t+C$', 'C': '$\\frac{4\\sqrt{3}}{9}t+\\frac{4\\sqrt{3}}{9}\\sin t\\cos t+C$', 'D': '$\\frac{8\\sqrt{3}}{9}t+\\frac{8\\sqrt{3}}{9}\\sin t\\cos t+C$', 'answer': 'C', 'explanation': ''}


In [ ]:
import pandas as pd

In [ ]:
train_df = train_dataset.to_pandas()
train_stat = train_df["question"].str.len().describe(percentiles=[0.25, 0.5, 0.75, 0.90, 0.95, 0.99])
print(train_stat)

In [5]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
import numpy as np
import re

2025-09-25 08:59:27.878747: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758790768.085496      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758790768.141507      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
model_name = "Qwen/Qwen3-4B-Instruct-2507"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map="cuda"
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

In [7]:
tokenizer.pad_token = tokenizer.eos_token
lora_config = LoraConfig(
    r=16,  
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 5,898,240 || all params: 4,028,366,336 || trainable%: 0.1464


In [8]:
def preprocess_function(examples):
    texts = []
    label_ids = []  # token ID 列表
    for q, a, b, c, d, ans in zip(
        examples['question'], examples['A'], examples['B'], examples['C'], examples['D'], examples['answer']
    ):
        prompt = f"以下是选择题（可多选）：\n问题：{q}\nA: {a}\nB: {b}\nC: {c}\nD: {d}\n请输出正确答案（多个用逗号分隔）："
        response = f"{ans}"
        full_text = tokenizer.apply_chat_template(
            [{"role": "user", "content": prompt}, {"role": "assistant", "content": response}],
            tokenize=False
        )
        texts.append(full_text)

        tokenized_ans = tokenizer.encode(ans.replace(',', ''), add_special_tokens=False)
        label_ids.append(tokenized_ans)

    return {"text": texts, "labels": label_ids}

tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
tokenized_val_dataset = validation_dataset.map(preprocess_function, batched=True, remove_columns=validation_dataset.column_names)


Map:   0%|          | 0/12342 [00:00<?, ? examples/s]

Map:   0%|          | 0/1346 [00:00<?, ? examples/s]

In [10]:
print(train_dataset[0])
print(validation_dataset[0])
print(tokenized_train_dataset[0])
print(torch.cuda.is_bf16_supported())

{'id': 0, 'question': '下列关于资本结构理论的说法中，不正确的是____。', 'A': '代理理论、权衡理论、有企业所得税条件下的MM理论，都认为企业价值与资本结构有关', 'B': '按照优序融资理论的观点，考虑信息不对称和逆向选择的影响，管理者偏好首选留存收益筹资，然后是发行新股筹资，最后是债务筹资', 'C': '权衡理论是对有企业所得税条件下的MM理论的扩展', 'D': '代理理论是对权衡理论的扩展', 'answer': 'B', 'explanation': ''}
{'id': 0, 'question': '下列关于税法基本原则的表述中，不正确的是____。', 'A': '税收法定原则包括税收要件法定原则和税务合法性原则', 'B': '税收公平原则源于法律上的平等性原则', 'C': '税收效率原则包含经济效率和行政效率两个方面', 'D': '税务机关按法定程序依法征税，可以自由做出减征、停征或免征税款的决定', 'answer': 'D', 'explanation': ''}
{'text': '<|im_start|>user\n以下是选择题（可多选）：\n问题：下列关于资本结构理论的说法中，不正确的是____。\nA: 代理理论、权衡理论、有企业所得税条件下的MM理论，都认为企业价值与资本结构有关\nB: 按照优序融资理论的观点，考虑信息不对称和逆向选择的影响，管理者偏好首选留存收益筹资，然后是发行新股筹资，最后是债务筹资\nC: 权衡理论是对有企业所得税条件下的MM理论的扩展\nD: 代理理论是对权衡理论的扩展\n请输出正确答案（多个用逗号分隔）：<|im_end|>\n<|im_start|>assistant\nB<|im_end|>\n', 'labels': [33]}
True


In [24]:
print(tokenized_train_dataset[0]['labels'].shape)

AttributeError: 'list' object has no attribute 'shape'

In [9]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predicted_ids = np.argmax(predictions, axis=-1)
    decoded_preds = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    parsed_preds = []
    parsed_labels = []
    for pred, label in zip(decoded_preds, decoded_labels):
        pred_set = set(re.findall(r'[A-D]', pred.strip()))
        label_set = set(re.findall(r'[A-D]', label.strip()))
        parsed_preds.append(pred_set)
        parsed_labels.append(label_set)
    

    accuracy = np.mean([p == l for p, l in zip(parsed_preds, parsed_labels)])
    return {"accuracy": accuracy}

In [27]:
training_args = TrainingArguments(
    output_dir="./qwen3_lora_finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    eval_strategy="steps",
    eval_steps=500,
    learning_rate=1e-4, 
    weight_decay=0.01,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=500,
    load_best_model_at_end=True,
    bf16=True,
    optim="adamw_torch",
    report_to="none",
)

collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    data_collator=collator,
    #peft_config=lora_config,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()
trainer.save_model("./qwen3_lora_finetuned")

Step,Training Loss,Validation Loss
